<a href="https://colab.research.google.com/github/rdrayef/Accidental.ly/blob/master/Comments_tfid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [79]:
# Step 2: Load and Preprocess Data
data = pd.read_csv('amazon_alexa.tsv', delimiter='\t')

In [80]:
# Extract the relevant columns
comments = data['verified_reviews'].astype(str)
labels = data['feedback']

In [81]:
# Perform data preprocessing (e.g., cleaning, tokenization) on the reviews as needed

# Step 4: TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
features = tfidf_vectorizer.fit_transform(comments)


In [82]:
# Convert labels to numpy array
labels = np.array(labels)

In [83]:
# Step 3: Split the Data
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

In [84]:
# Reshape the input features for LSTM
train_features = train_features.toarray().reshape(train_features.shape[0], 1, train_features.shape[1])
test_features = test_features.toarray().reshape(test_features.shape[0], 1, test_features.shape[1])


In [85]:
# Step 5: Build LSTM Model
model = Sequential()
model.add(LSTM(128, input_shape=(train_features.shape[1], train_features.shape[2]), dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [86]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 6: Train the Model
early_stopping = EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)
history = model.fit(train_features, train_labels, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/10
63/63 [==============================] - 5s 47ms/step - loss: 0.5353 - accuracy: 0.9132 - val_loss: 0.3512 - val_accuracy: 0.9345
Epoch 2/10
63/63 [==============================] - 3s 41ms/step - loss: 0.3000 - accuracy: 0.9177 - val_loss: 0.2339 - val_accuracy: 0.9345
Epoch 3/10
63/63 [==============================] - 3s 43ms/step - loss: 0.2412 - accuracy: 0.9177 - val_loss: 0.2063 - val_accuracy: 0.9345
Epoch 4/10
63/63 [==============================] - 4s 62ms/step - loss: 0.2005 - accuracy: 0.9182 - val_loss: 0.1851 - val_accuracy: 0.9345
Epoch 5/10
63/63 [==============================] - 3s 42ms/step - loss: 0.1685 - accuracy: 0.9211 - val_loss: 0.1655 - val_accuracy: 0.9365
Epoch 6/10
63/63 [==============================] - 3s 41ms/step - loss: 0.1364 - accuracy: 0.9365 - val_loss: 0.1487 - val_accuracy: 0.9405
Epoch 7/10
63/63 [==============================] - 3s 41ms/step - loss: 0.1068 - accuracy: 0.9563 - val_loss: 0.1376 - val_accuracy: 0.9425
Epoch 8/10
63

In [87]:
# Example comments for testing
example_comments = [
    "Great	",
    "looks great",
    "Not much features."
]

# Preprocess the example comments (cleaning, tokenization) as needed

# Vectorize the example comments using TF-IDF
example_features = tfidf_vectorizer.transform(example_comments)
example_features = example_features.toarray().reshape(example_features.shape[0], 1, example_features.shape[1])

# Predict the sentiment of the example comments
predictions = model.predict(example_features)

# Convert the predictions to sentiment labels
sentiments = ['Positive' if prediction >= 0.5 else 'Negative' for prediction in predictions]

# Print the sentiment predictions for the example comments
for comment, sentiment in zip(example_comments, sentiments):
    print("Comment:", comment)
    print("Sentiment:", sentiment)

1/1 [==============================] - 0s 265ms/step
Comment: Great	
Sentiment: Positive
Comment: looks great
Sentiment: Positive
Comment: Not much features.
Sentiment: Negative
